# Mini-Project: JSON-Based Data Exercise


#### Andy Harless

### Imports for Python, Pandas

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

### Question 1

#### Read the data

In [2]:
top_json_df = pd.read_json('data/world_bank_projects.json')

#### Make sure each entry appears to represent a unique project

In [3]:
ids = top_json_df['_id'].values
print('Number of ID dicts: ', len(ids))
print('Number of ID dicts with $oid keys: ', len([id['$oid'] for id in ids]))
print('Total number of keys in ID dicts: ', sum([len(id) for id in ids]))
print('Number of unique OIDs: ', len({id['$oid'] for id in ids}))

Number of ID dicts:  500
Number of ID dicts with $oid keys:  500
Total number of keys in ID dicts:  500
Number of unique OIDs:  500


#### Make sure country identification is consistent across identifiers

In [4]:
country_data = top_json_df[['country_namecode','countryname','countrycode','countryshortname']]
country_data.count()

country_namecode    500
countryname         500
countrycode         500
countryshortname    500
dtype: int64

In [5]:
country_data.nunique()

country_namecode    118
countryname         118
countrycode         118
countryshortname    118
dtype: int64

In [6]:
country_data.groupby('country_namecode').nunique().max()

country_namecode    1
countryname         1
countrycode         1
countryshortname    1
dtype: int64

#### Find the 10 countries with the most projects

In [7]:
country_data.countryshortname.value_counts().head(10)

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Nepal                 12
Morocco               12
Africa                11
Mozambique            11
Name: countryshortname, dtype: int64

### Question 3 (which must be answered before question 2, since I will use the result)

#### Get theme data

In [8]:
json_data=json.load((open('data/world_bank_projects.json')))
themes = json_normalize(json_data, 'mjtheme_namecode')
themes.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


#### Examine the data

In [9]:
themes.count()

code    1499
name    1499
dtype: int64

In [10]:
themes[themes.name==''].count()

code    122
name    122
dtype: int64

#### Generate dictionary mapping theme codes to names

In [11]:
theme_names = themes[themes.name!=''].drop_duplicates().set_index('code').name.to_dict()

#### Replace empty names with correct names

In [12]:
nameless = themes.name==''
replacement_names = [theme_names[c] for c in themes[nameless].code] #.values
themes.loc[nameless,'name'] = replacement_names

In [13]:
themes.head()

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


### Question 2 (See question 3 above.)

#### Display top10 major project themes

In [14]:
themes.name.value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64